In [1]:
# HCASE Embedding Reduced
#

# Author: Gergely Zahoranszky-Kohalmi, PhD
#
# Email: gergely.zahoranszky-kohalmi@nih.gov
#
# Organization: National Center for Advancing Translational Sciences (NCATS/NIH)
#
# Workflow:

# 1. Select 90% of reference scaffolds.
# 2. Merge the resultant set with cehrry-picked scaffold, and deduplicate (this is to assure consistency of analysis).
# 3. Perform HCASE of drugs in reduced ChEMBL space
# 4. Generate plots for Cherry-picked scaffolds on each HCASE embedding.
# 5. Generate comparative plot between HCASE embeddings of drugs in full and reduced ChEBML space
# on the largest zommon z parameter.


In [2]:
import hcase
import pandas as pd

In [3]:
# Config section



fname_reference_scaffolds = '../data/hc_space.tab'
fname_cherry_picked_scaffolds = '../data/cherrypicked_scaffolds.tab'
fname_structures = '../data/STD_drugbank_approved_structures_v5.txt'

fname_out_reduced_reduced_ref_scaffolds = '../data/red_ref_scaffolds.tab'
fname_out_space = '../data/red_hc_space.tab'
fname_out_embedding = '../data/red_drugs_emb_hcase_chembl.tab'

n_dim = 2



use_precomputed_reference_spaces = False

rnd_seed_reduced = 12345
reduced_fraction = 0.9

do_downsampling = False
rnd_seed = 55555
sample_size = 200




In [4]:
# Import datasets

df_ref_scaffolds = pd.read_csv (fname_reference_scaffolds, sep = '\t')
df_cp = pd.read_csv (fname_cherry_picked_scaffolds, sep = '\t')
df_structures = pd.read_csv (fname_structures, sep = '\t')

print (df_ref_scaffolds.head)
print (df_cp.head)

<bound method NDFrame.head of                                                structure  order  \
0                                                C=C1CC1      1   
1                                                  C1CN1      2   
2                                             C1C[NH2+]1      3   
3                                                  C1CS1      4   
4                                                 C1=CC1      5   
...                                                  ...    ...   
55592  O=C1CCSSC[C@@H]2NC(=O)[C@@H]3CSSC[C@@H](C(=O)N...  55593   
55593  O=C(NCC(=O)N1Cc2ccccc2C[C@H]1C(=O)N1CC2CCCCC2[...  55594   
55594  S=P(N/N=C/c1ccc(OP2(Oc3ccc(/C=N/NP(=S)(Oc4cccc...  55595   
55595  O=C(CNC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)...  55596   
55596  O=C(CNC(=O)[C@@H]1C2CCCCC2CN1C(=O)[C@@H]1Cc2cc...  55597   

           scaffold_id                                       scaffold_key  
0          scaffold.10  3 0 0 1 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...  
1           s

In [5]:
# Clean-up datasets

df_ref_scaffolds = df_ref_scaffolds[['structure', 'scaffold_id', 'scaffold_key']].copy()

df_cp = df_cp[['structure', 'scaffold_id', 'scaffold_key', 'color']].copy()



df_structures = df_structures.rename (columns = {'Structure': 'structure', 'ID': 'id'})

In [6]:
# Functions

def smiles2inchikey (smiles):
    if smiles == 'NA':
        inchi = 'NA'
    else:
        try:
            mol = Chem.MolFromSmiles(smiles)
            inchi = Chem.MolToInchi (mol)
        except:
            inchi = 'NA'

    if inchi == 'NA':
        inchikey = 'NA'
    else:
        try:
            inchikey = Chem.InchiToInchiKey (inchi)
        except:
            inchikey = 'NA'
    
    return (inchikey)    



def generate_reduced_reference_scaffold_set (df_scaffolds_orig, df_cp_scaffolds, reduced_fraction, rnd_seed_reduced):
   
    # Reduce reference set to 90%, randomly
    df_ref_scaffolds = df_scaffolds_orig.sample(frac = reduced_fraction, random_state = rnd_seed_reduced)

    # Merge with Cherry-Picked scaffolds to maintain consistency so that the embeddings of the
    # Cherry-Picked scasffolds can be compared in the original and in the reduced space
    df_ref_scaffolds = df_ref_scaffolds[['structure', 'scaffold_id', 'scaffold_key']].copy()
    

    #df_cp_scaffolds = df_cp_scaffolds[['structure', 'scaffold_id', 'scaffold_key', 'color']].copy()
    
    df_cp_scaffolds_orig = df_cp_scaffolds.copy()
    df_cp_scaffolds = df_cp_scaffolds_orig[['structure', 'scaffold_id', 'scaffold_key']].copy()
    df_cp_color = df_cp_scaffolds_orig[['scaffold_id', 'color']].copy()

    
    
    # append reduced scaffold set and cherry-picked scaffolds
    df = df_ref_scaffolds.append (df_cp_scaffolds, ignore_index = True)
    
    
    # deduplicate sets

    df = df.groupby (['scaffold_id'], as_index = False).agg('first')

    df = df.reset_index(drop = True)
    
    
    
    # split deduplicated set into cherry-picked and not cherry-picked partitions

    df_in_cp = df[df['scaffold_id'].isin(list(df_cp_scaffolds['scaffold_id']))].copy()
   
    
    df_not_in_cp = df[~df['scaffold_id'].isin(list(df_cp_scaffolds['scaffold_id']))].copy()



    # assign colors: assign color -1 to non-cherry-picked partition

    
    df_in_cp = df_in_cp.merge (df_cp_color, on = 'scaffold_id', how = 'inner')
    df_not_in_cp['color'] = -1   
    
    
    #df_in_cp = df_in_cp.astype({'color': int})

 
    

    # append cherry-picked and non-cherry-picked partitions


    df = df_in_cp.append (df_not_in_cp, ignore_index = True)
    df = df[['structure', 'scaffold_id', 'scaffold_key', 'color']].copy()



    
    return (df)


In [7]:
# Reduce reference scaffold set

# Reduce reference set to 90%, randomly
df_ref_scaffolds = generate_reduced_reference_scaffold_set (df_ref_scaffolds, df_cp, reduced_fraction, rnd_seed_reduced)

    
# Data structure adjustment

# This is just to make the data structure compliant which HCASE which at this point expects a set of scaffolds,
# but here a set if compounds are provided. It's not an issue in the workflow, as the train method of hcase 
# actually generates the unique BM-scaffold set from the compounds. 
df_ref_scaffolds['hash'] = df_ref_scaffolds.apply (lambda x: smiles2inchikey(x['structure']), axis = 1)
df_ref_scaffolds['ptype'] = 'scaffold'


df_ref_scaffolds.to_csv (fname_out_reduced_reduced_ref_scaffolds, sep = '\t', index = False)


df_ref_scaffolds = df_ref_scaffolds.rename (columns = {'scaffold_id': 'pattern_id'})

In [8]:
# Reduce datasets for workflow development (optional)

if do_downsampling:

    df_ref_scaffolds = df_ref_scaffolds.sample (n = sample_size, random_state = rnd_seed)
    df_structures = df_structures.sample (n = sample_size, random_state = rnd_seed)



In [9]:
# Create ChEMBL Scaffold space
print (df_ref_scaffolds.head)

if not use_precomputed_reference_spaces:
    
    df_hcase_space = hcase.train (df_ref_scaffolds)
    df_hcase_space.to_csv (fname_out_space, sep = '\t', index = False)

else:
    
    df_hcase_space = pd.read_csv (fname_out_space, sep = '\t')


<bound method NDFrame.head of                                         structure       pattern_id  \
0                  O=C(CCc1ccccc1)NCC(=O)N1CCCCC1  scaffold.100896   
1                  O=C(CNC(=O)c1ccccc1)NCC1CCCCC1  scaffold.100938   
2             O=C(N/C=C/c1ccccc1)NC1C(=O)N2CCSC12  scaffold.101000   
3                O=C(NC(=O)c1ccccc1)NC1CN2CCC1CC2  scaffold.101307   
4               O=C1Nc2ccccc2C(=O)/C1=C/Nc1ccccc1  scaffold.102134   
...                                           ...              ...   
50114                        O=C1CCCC(c2ccccc2)N1    scaffold.9995   
50115  c1ccc([C@H]2Nc3ccccc3[C@H]3NCC[C@@H]23)cc1   scaffold.99954   
50116                 c1ccc(CCc2nccn2Cc2cccs2)cc1   scaffold.99979   
50117                 c1ccc(CCc2nccn2Cc2ccco2)cc1   scaffold.99980   
50118                         O=C1CCC=NN1c1ccccc1    scaffold.9999   

                                            scaffold_key  color hash     ptype  
0      18 6 7 2 0 6 1 0 4 0 0 0 4 4 0 0 2 2 0 0 

[05:55:29] WARNING: Omitted undefined stereo
[05:55:29] WARNING: Omitted undefined stereo
[05:55:29] WARNING: Omitted undefined stereo
[05:55:30] WARNING: Proton(s) added/removed
[05:55:30] WARNING: Proton(s) added/removed
[05:55:30] WARNING: Omitted undefined stereo
[05:55:30] WARNING: Omitted undefined stereo
[05:55:30] WARNING: Proton(s) added/removed
[05:55:30] WARNING: Proton(s) added/removed
[05:55:30] WARNING: Proton(s) added/removed
[05:55:30] WARNING: Proton(s) added/removed
[05:55:30] WARNING: Omitted undefined stereo
[05:55:30] WARNING: Proton(s) added/removed
[05:55:30] WARNING: Omitted undefined stereo
[05:55:30] WARNING: Proton(s) added/removed
[05:55:30] WARNING: Omitted undefined stereo
[05:55:30] WARNING: Proton(s) added/removed; Omitted undefined stereo
[05:55:30] WARNING: Proton(s) added/removed
[05:55:30] WARNING: Proton(s) added/removed
[05:55:30] WARNING: Omitted undefined stereo
[05:55:30] WARNING: Proton(s) added/removed
[05:55:30] WARNING: Omitted undefined ste

[*] Number of scaffolds in input:
50119
[*] Number of unique reference scaffolds:
50119


In [10]:
# Embed structures into HCASE space


    
df_embedded = hcase.embed (df_hcase_space, df_structures, n_dim)




[07:32:05] Explicit valence for atom # 21 N, 4, is greater than permitted
[07:32:05] Explicit valence for atom # 1 N, 4, is greater than permitted
[07:32:05] Explicit valence for atom # 6 N, 4, is greater than permitted
[07:32:05] Explicit valence for atom # 1 N, 4, is greater than permitted
[07:32:05] Explicit valence for atom # 12 N, 4, is greater than permitted
[07:32:05] Explicit valence for atom # 2 N, 4, is greater than permitted
[07:32:05] Explicit valence for atom # 3 O, 3, is greater than permitted
[07:32:06] Explicit valence for atom # 17 N, 4, is greater than permitted
[07:32:06] Explicit valence for atom # 11 N, 4, is greater than permitted
[07:32:06] Explicit valence for atom # 1 N, 4, is greater than permitted
[07:32:06] Explicit valence for atom # 2 N, 4, is greater than permitted
[07:32:06] Explicit valence for atom # 20 N, 4, is greater than permitted
[07:32:06] Explicit valence for atom # 17 N, 4, is greater than permitted
[07:32:06] Explicit valence for atom # 25 N, 

[WARNING] SMILES: O=c1nc[nH]c2c1NC(CNc1ccccc1)CN2 cannot be processed by RDKit.
[WARNING] SMILES: O=c1nc2[nH]c3ccccc3nc-2c(=O)[nH]1 cannot be processed by RDKit.
[WARNING] SMILES: O=c1nc[nH]c2ncc(CNc3ccccc3)nc12 cannot be processed by RDKit.
[WARNING] SMILES: O=c1[nH]c(=O)c2[nH]cnc2[nH]1 cannot be processed by RDKit.
[WARNING] SMILES: O=c1nc(-c2cccc(S(=O)(=O)N3CCNCC3)c2)[nH]c2cn[nH]c12 cannot be processed by RDKit.


[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] Can't kekulize mol.  Unkekulized atoms: 1 3 5 7 8 9

[07:32:07] SMILES Parse Error: syntax error for input: 'NA'
[07:32:07] Can't kekulize mol.  Unkekulized atoms: 1 2 3 20 21 22 24

[07:32:07] SMILES Parse Error: syntax error for input: 'NA'
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] Can't kekulize mol.  Unkekulized atoms: 1 2 3 8 9 10 20 21 22

[07:32:07] SMILES Parse Error: syntax error for input: 'NA'
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] WARNING: Omitted undefined stereo


[WARNING] SMILES: O=c1ccn(C2CC2)c2cc(N3C[C@@H]4CCCN[C@@H]4C3)ccc12 cannot be processed by RDKit.
[WARNING] SMILES: O=c1ccc(-c2ccncc2)c[nH]1 cannot be processed by RDKit.
[WARNING] SMILES: O=c1ccn(C2CCCO2)c(=O)[nH]1 cannot be processed by RDKit.


[07:32:07] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 11

[07:32:07] SMILES Parse Error: syntax error for input: 'NA'
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] Can't kekulize mol.  Unkekulized atoms: 1 2 3 10

[07:32:07] SMILES Parse Error: syntax error for input: 'NA'
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] Can't kekulize mol.  Unkekulized atoms: 1 3 4 5 6 7 8 9 10

[07:32:07] SMILES Parse Error: syntax error for input: 'NA'
[07:32:07] WARNING: Omitted undefined stereo
[07:32:07] Can't kekulize mol.  Unkekulized atoms: 1 3 5 7 8 9

[07:32:07] SMILES Parse Error: syntax error for input: 'NA'


...

[WARNING] SMILES: O=c1cc[nH]c(=O)[nH]1 cannot be processed by RDKit.
[WARNING] SMILES: N=C1CCCN1Cc1cc(=O)[nH]c(=O)[nH]1 cannot be processed by RDKit.
[WARNING] SMILES: O=c1nc[nH]c2c1NC(CNc1ccccc1)CN2 cannot be processed by RDKit.
[WARNING] SMILES: O=c1cc(-c2ccccc2)oc2ccccc12 cannot be processed by RDKit.


[07:32:15] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5

[07:32:15] SMILES Parse Error: syntax error for input: 'NA'
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] Can't kekulize mol.  Unkekulized atoms: 7 8 9 12

[07:32:16] SMILES Parse Error: syntax error for input: 'NA'
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] WARNING: Metal was disconnected; Proton(s) added/removed
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6

[07:32:16] SMILES Parse Error: syntax error for input: 'NA'
[07:32:16] Can't kekulize mol.  Unkekulized atoms: 1 2 3 11 12 13 14 15 16

[07:32:16] SMILES Parse Error: syntax error for input: 'NA'
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] WARNING: Omitted 

[WARNING] SMILES: O=c1cccc2c(c1)CCCc1cc(OC3CCCCO3)ccc1-2 cannot be processed by RDKit.
[WARNING] SMILES: O=c1nc[nH]c2c1NC(CNc1ccccc1)CN2 cannot be processed by RDKit.
[WARNING] SMILES: O=c1ccn(-c2ccccn2)c2cc(N3CCC3)ccc12 cannot be processed by RDKit.


[07:32:16] Can't kekulize mol.  Unkekulized atoms: 1 2 3 11 12 13 18 19 20

[07:32:16] SMILES Parse Error: syntax error for input: 'NA'
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 6 14 15 16

[07:32:16] SMILES Parse Error: syntax error for input: 'NA'
[07:32:16] WARNING: Omitted undefined stereo
[07:32:16] WARNING: Omitted undefined stereo


[WARNING] SMILES: O=c1ccc2cc(OC3CCCCO3)ccc2o1 cannot be processed by RDKit.
[*] Number of input structures: 1623
[*] Number of structures for which scaffold_key was generated: 1474
[*] Finding closest reference scaffold for structure 1 out of 1474 .
[*] Finding closest reference scaffold for structure 2 out of 1474 .
[*] Finding closest reference scaffold for structure 3 out of 1474 .
[*] Finding closest reference scaffold for structure 4 out of 1474 .
[*] Finding closest reference scaffold for structure 5 out of 1474 .
[*] Finding closest reference scaffold for structure 6 out of 1474 .
[*] Finding closest reference scaffold for structure 7 out of 1474 .
[*] Finding closest reference scaffold for structure 8 out of 1474 .
[*] Finding closest reference scaffold for structure 9 out of 1474 .
[*] Finding closest reference scaffold for structure 10 out of 1474 .
[*] Finding closest reference scaffold for structure 11 out of 1474 .
[*] Finding closest reference scaffold for structure 12 ou

In [11]:
df_embedded.to_csv (fname_out_embedding, sep = '\t', index = False)

In [12]:
# References

# Ref: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html